In [1]:
from get_training_data import *

import tensorflow as tf
import keras
import pickle
import gzip
import os
import random
import matplotlib.pyplot as plt
%matplotlib nbagg

/home/andy/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with gzip.open('train_cases.pkl.gz','rb') as fp:
    train_cases = pickle.load(fp)
type(train_cases), len(train_cases)

(tuple, 2)

In [3]:
train_X = train_cases[0]
train_y = train_cases[1]
ntrain = train_X.shape[0]
type(train_X), train_X.shape, train_X.dtype, type(train_y), train_y.shape, train_y.dtype

(numpy.ndarray,
 (16384, 16, 16, 128),
 dtype('float32'),
 numpy.ndarray,
 (16384, 1),
 dtype('float64'))

In [4]:
with gzip.open('valid_cases.pkl.gz','rb') as fp:
    valid_cases = pickle.load(fp)
valid_X = valid_cases[0]
valid_y = valid_cases[1]
nvalid = valid_X.shape[0]
type(valid_X), valid_X.shape, valid_X.dtype, type(valid_y), valid_y.shape, valid_y.dtype

(numpy.ndarray,
 (4096, 16, 16, 128),
 dtype('float32'),
 numpy.ndarray,
 (4096, 1),
 dtype('float64'))

In [5]:
def choose_training_case():
    i = random.randint(0, ntrain-1)
    return(train_X[i,:,:,:], train_y[i,:])

def choose_validation_case():
    i = random.randint(0, nvalid-1)
    return(valid_X[i,:,:,:], valid_y[i,:])

In [6]:
ADD_CONST = 1e-5
CENTER = -2.5
SCALE = 5
XFORM = True

In [7]:
def preproc(a):
    if XFORM:
        return((np.log(a+ADD_CONST) - CENTER) / SCALE)
    else:
        return(a)

In [8]:
def make_batch_from_megabatch(batchsize=32, validation=False):
    get_case = choose_validation_case if validation else choose_training_case    
    xbatch = np.empty([0, INPUTS_PER_BEAT, NBEATS, NCHANNELS],dtype=np.float32)
    ybatch = np.empty([0, 1])
    for i in range(batchsize):
        case = get_case()
        x = np.expand_dims(case[0], axis=0)
        xbatch = np.concatenate([xbatch, x], axis=0)
        y = np.array(int(case[1])).reshape([1,1])
        ybatch = np.concatenate([ybatch, y], axis=0)
    return(preproc(xbatch), ybatch)

def train_gen(batchsize=32):
    while True:
        yield(make_batch_from_megabatch(batchsize, validation=False))

def valid_gen(batchsize=32):
    while True:
        yield(make_batch_from_megabatch(batchsize, validation=True))

In [9]:
# The generators are way too slow: have to fix this
for t in train_gen():
    break
t[0].shape, t[1].shape

((32, 16, 16, 128), (32, 1))

In [10]:
for v in valid_gen():
    break
v[0].shape, v[1].shape

((32, 16, 16, 128), (32, 1))

In [11]:
input_shape = (INPUTS_PER_BEAT, NBEATS, NCHANNELS)

model = keras.Sequential()
model.add(keras.layers.Conv2D(64, 1, activation='relu', input_shape=input_shape))
model.add(keras.layers.SpatialDropout2D(.1))
model.add(keras.layers.Conv2D(32, 1, activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(12, (1,4), activation='relu'))
model.add(keras.layers.SpatialDropout2D(.3))
model.add(keras.layers.Conv2D(25, (4,6), activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(3, (1,4), activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(.7))
model.add(keras.layers.Dense(20))
model.add(keras.layers.PReLU())
model.add(keras.layers.Dropout(.1))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [12]:
model.compile(loss='binary_crossentropy', 
              optimizer=keras.optimizers.Adam(lr=.003,decay=2e-4), 
              metrics=['accuracy'])

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 64)        8256      
_________________________________________________________________
spatial_dropout2d_1 (Spatial (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        2080      
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 13, 12)        1548      
_________________________________________________________________
spatial_dropout2d_2 (Spatial (None, 16, 13, 12)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 8, 25)         7225      
__________

In [14]:
valid=valid_gen()
train=train_gen()
model.fit_generator(train, 
                    validation_data=valid, validation_steps=64,
                    epochs=60, 
                    steps_per_epoch=64)

Epoch 1/60
64/64 [==============================] - 2s 37ms/step - loss: 0.6319 - acc: 0.7437 - val_loss: 0.5869 - val_acc: 0.7734
Epoch 2/60
64/64 [==============================] - 1s 20ms/step - loss: 0.5709 - acc: 0.7764 - val_loss: 0.5552 - val_acc: 0.7671
Epoch 3/60
64/64 [==============================] - 1s 20ms/step - loss: 0.5699 - acc: 0.7588 - val_loss: 0.5550 - val_acc: 0.7700
Epoch 4/60
64/64 [==============================] - 1s 21ms/step - loss: 0.5471 - acc: 0.7603 - val_loss: 0.5553 - val_acc: 0.7686
Epoch 5/60
64/64 [==============================] - 1s 21ms/step - loss: 0.5273 - acc: 0.7661 - val_loss: 0.5539 - val_acc: 0.7598
Epoch 6/60
64/64 [==============================] - 1s 20ms/step - loss: 0.5067 - acc: 0.7598 - val_loss: 0.5168 - val_acc: 0.7729
Epoch 7/60
64/64 [==============================] - 1s 20ms/step - loss: 0.4761 - acc: 0.7661 - val_loss: 0.4953 - val_acc: 0.7720
Epoch 8/60
64/64 [==============================] - 1s 20ms/step - loss: 0.4456 - a

In [15]:
for i in range(10):
    song, tempo, compatible, clip = get_validation_case()
    print( song, tempo )
    c = clip_to_tf_input(resample_clip(clip))
    x = np.expand_dims(preproc(c), axis=0)
    p = model.predict(x)[0][0]
    print( 'Predicted: ', p, '   Actual: ', compatible, '\n' )

BloodyMerryMorning 112
Predicted:  0.32069367    Actual:  True 

EternalFlame 126
Predicted:  0.061221957    Actual:  False 

IThinkWereAloneNow2MP 129
Predicted:  0.9470875    Actual:  True 

SmoothOperator 96
Predicted:  1.04596424e-07    Actual:  False 

AHardRainsAGonnaFall 44
Predicted:  0.0017600155    Actual:  False 

SmoothOperator 68
Predicted:  2.6641322e-07    Actual:  False 

ChatanoogaChooChoo 235
Predicted:  0.12721808    Actual:  False 

WalkThisWay1 72
Predicted:  0.001208258    Actual:  False 

HoldMeTight 45
Predicted:  0.025826462    Actual:  False 

IllBeThereForYou 43
Predicted:  0.053643607    Actual:  False 

